In [1]:
#!/home/brian/anaconda3/bin/python
import sys,zipfile,os,shutil,time,wget,re
from pathlib import Path
from multiprocessing import Process, Queue, Pool
from datetime import date, timedelta


In [2]:
#!/home/brian/anaconda3/bin/python
import sys,zipfile,os,shutil,time,wget,re
from pathlib import Path
from multiprocessing import Process, Queue, Pool
from datetime import date, timedelta
def Program():
    ip1 = sys.argv[1]
    ip2 = sys.argv[2]
    ip3 = sys.argv[3]
    ip4 = sys.argv[4]
    if __name__ == "__main__":

        start_time = time.time()
        
        print(' \n Downloading Player CSV \n')
        
        p = 'https://www.retrosheet.org/retroID.htm'
        wget.download(p, 'player.csv')

        with open('player.csv', encoding='cp1252') as f:
            lines = f.readlines()
        r = "(\w+,\w+,\w+,\w+/\S+)"
        match = re.finditer(r,str(lines))
        
        print(' \n Parsing Player CSV \n')
        
        with open("players.csv", "a") as f1:
            f1.write('ID'+','+'Last'+','+'First'+','+'Play_debut'+','+'Mgr_debut'+','+'Coach_debut'+','+'Ump_debut'+"\n")
            for m in match:
                f1.write(m.group(1) + '\n') 
        os.remove('player.csv')
        
        print('\n Downloading Lahman Zip \n')
        
        R =  Path('./Lahman')
        if R.exists() is False:
            os.makedirs('Lahman')
        os.chdir("./Lahman")
        p = 'https://github.com/chadwickbureau/baseballdatabank/archive/v2019.2.zip'
        try: 
            wget.download(p, 'lahman.zip')
            zipfile.ZipFile('lahman.zip').extractall('.')
            #os.rename('baseballdatabank-2019.2', 'Lahman')
            os.remove('lahman.zip')
        except:
            pass
        os.chdir('..')
        
        print('\n Downloading Retrosheets zips \n ')
        R =  Path('./Retrosheets/Roster')
        if R.exists() is False:
            os.makedirs('Retrosheets/Roster')
        T =  Path('./Retrosheets/Team')
        if T.exists() is False:
            os.makedirs('Retrosheets/Team')
        E =  Path('./Retrosheets/Events')
        if E.exists() is False:
            os.makedirs('Retrosheets/Events')
        try:   
            shutil.move('players.csv', './Retrosheets')
        except:
            pass

        for i in range(1900,2020):
            try: 
                year = i
                p = 'https://www.retrosheet.org/events/' + str(year) + 'eve.zip'
                wget.download(p)
                print('\n Downloading Regular Season: ' + str(year) )
                zipfile.ZipFile(str(year) + 'eve.zip').extractall('.')
                for file in os.listdir("./"):
                    try:
                        if file.endswith(".ROS"):
                            shutil.move(file, R)
                        elif file.endswith(str(year)):
                            shutil.move(file, T)
                        elif file.endswith(".EVN"):
                            shutil.move(file, E)
                        elif file.endswith(".EVA"):
                            os.remove(file)
                        elif file.endswith("EVN"):
                            os.remove(file)
                        elif file.endswith("EDA"):
                            os.remove(file)
                        elif file.endswith("EDN"):
                            os.remove(file)
                        elif file.endswith("zip"):
                            os.remove(file)
                    except:
                        os.remove(file)
            except:
                pass

            for i in range(1900,2020):
                try: 
                    year = i
                    p = 'https://www.retrosheet.org/events/' + str(year) + 'post.zip'
                    wget.download(p)
                    print('\n Downloading Playoffs: ' + str(year) )
                    zipfile.ZipFile(str(year) + 'post.zip').extractall('.')
                    for file in os.listdir("./"):
                        try:

                            if file.endswith(".EVE"):
                                shutil.move(file, E)
                            elif file.endswith(str(year)):
                                os.remove(file)
                            elif file.endswith("ROS"):
                                os.remove(file)
                            elif file.endswith("zip"):
                                os.remove(file)
                        except:
                            os.remove(file)
                except:
                    pass
        def get_GIDs(q):
            '''
            Function to get GameIDs 
            Input: Queue of dates 
            Output: Queue and dictionary of the Gids  
            Note: Make sure the function of this directory is located in the 
            '''

            day = q.get()
            All_urls = []
            Return_URl = []
            while (day != 'Done'):
                All_urls.append('./MLB_Gameday_Data/year_' + str(day[0:4]) + '/month_'+ str(day[5:7]) + '/day_' + str(day[8:10]))      
                day = q.get()
            ID_dic = {}
            GID_queue = Queue()

            r = '(\w+)/\S\S(\w+_[0-9]{4}_[0-9]{2}_[0-9]{2}_\w+_\w+_[0-9])'
            for i in range(0,len(All_urls)):
                with open(All_urls[i] + '/index.html', 'r') as f:
                    try:
                        lines = f.readlines()
                        matches = re.finditer(r,str(lines))
                        for m in matches:
                            GID_queue.put(m.group(2))
                            ID_dic[m.group(1)] = m.group(2)
                            Return_URl.append(All_urls[i])
                    except:
                        pass
            GID_Search(ID_dic, GID_queue, All_urls)



        def GID_Search(Dict, GID, url):
            '''
            Function to ccheck GID and game values
            Input: Dictionary of game values and GIDs, a queue of GIDs 
            Output: Creation of a Directory with all the GIDs in them
            Note: Make sure the function of this directory is located in the 

            let us make url a list too

            '''
            home_dir = os.getcwd()
            MLB_GIDs =  Path('./MLB_GIDs')
            if MLB_GIDs.exists() is False:
                os.makedirs('./MLB_GIDs')
            Move_file = home_dir + '/MLB_GIDs'
            GID.put('Done')
            GameID = GID.get()
            urlNumber = 0
            while (GameID != 'Done' and urlNumber < len(url)):

                os.chdir(home_dir)
                os.chdir(url[urlNumber])
                #print(urlNumber)
                list_dir = os.listdir()
                
                urlNumber += 1
                try:
                    for i in list_dir:
                        Game_numbers = []
                        if i != 'index.html':
                            Game_numbers.append(i)
                            
                        for i in Game_numbers:
                            File_name = Dict.get(i)
                            try:
                                shutil.copytree(i, Move_file + '/' + File_name)
                            except:
                                pass
                                #print('failed to move:' + i)


                except:
                    pass


        def run_action(year1,month1,day1,threads):   
            #start_time = time.time()
            Daysqueue = Queue()

            os.chdir('/home/brian/Data3402/Project1')
            start = date(year1,month1,day1)
            today = date.today()
            day = start
            while(day <= today):
                Daysqueue.put(day.isoformat())
                day += timedelta(1)
            Daysqueue.put('Done')

            num_threads = threads
            thrds = []
            for i in range(num_threads):
                p = Process(target = get_GIDs, args=[Daysqueue])
                thrds.append(p)
                p.start()

                #wait for threads to finish
            for p in thrds:
                p.join
            elapsed_6 = time.time()-start_time
            print("Execution completed in {} seconds".format(elapsed_6))
    run_action(inp1,inp2,inp3,inp4)

In [3]:
Program()

IndexError: list index out of range

In [48]:
        def get_GIDs(q):
            '''
            Function to get GameIDs 
            Input: Queue of dates 
            Output: Queue and dictionary of the Gids  
            Note: Make sure the function of this directory is located in the 
            '''

            day = q.get()
            All_urls = []
            Return_URl = []
            while (day != 'Done'):
                All_urls.append('./MLB_Gameday_Data/year_' + str(day[0:4]) + '/month_'+ str(day[5:7]) + '/day_' + str(day[8:10]))      
                day = q.get()
            ID_dic = {}
            GID_queue = Queue()

            r = '(\w+)/\S\S(\w+_[0-9]{4}_[0-9]{2}_[0-9]{2}_\w+_\w+_[0-9])'
            for i in range(0,len(All_urls)):
                with open(All_urls[i] + '/index.html', 'r') as f:
                    try:
                        lines = f.readlines()
                        matches = re.finditer(r,str(lines))
                        for m in matches:
                            GID_queue.put(m.group(2))
                            ID_dic[m.group(1)] = m.group(2)
                            Return_URl.append(All_urls[i])
                    except:
                        pass
            GID_Search(ID_dic, GID_queue, All_urls)



        def GID_Search(Dict, GID, url):
            '''
            Function to ccheck GID and game values
            Input: Dictionary of game values and GIDs, a queue of GIDs 
            Output: Creation of a Directory with all the GIDs in them
            Note: Make sure the function of this directory is located in the 

            let us make url a list too

            '''
            home_dir = os.getcwd()
            MLB_GIDs =  Path('./MLB_GIDs')
            if MLB_GIDs.exists() is False:
                os.makedirs('./MLB_GIDs')
            Move_file = home_dir + '/MLB_GIDs'
            GID.put('Done')
            GameID = GID.get()
            urlNumber = 0
            while (GameID != 'Done' and urlNumber < len(url)):

                os.chdir(home_dir)
                os.chdir(url[urlNumber])
                #print(urlNumber)
                list_dir = os.listdir()
                urlNumber += 1
                try:
                    for i in list_dir:
                        Game_numbers = []
                        if i != 'index.html':
                            Game_numbers.append(i)
                        for i in Game_numbers:
                            File_name = Dict.get(i)

                            shutil.copytree(i, Move_file + '/' + Dict.get(i))

                except:
                    pass


        def run_action(year1,month1,day1,threads):   

            Daysqueue = Queue()

            os.chdir('/home/brian/Data3402/Project1')
            start = date(year1,month1,day1)
            today = date.today()
            day = start
            while(day <= today):
                Daysqueue.put(day.isoformat())
                day += timedelta(1)
            Daysqueue.put('Done')

            num_threads = threads
            thrds = []
            for i in range(num_threads):
                p = Process(target = get_GIDs, args=[Daysqueue])
                thrds.append(p)
                p.start()

                #wait for threads to finish
            for p in thrds:
                p.join
            elapsed_6 = time.time()-start_time
            print("Execution completed in {} seconds".format(elapsed_6))
    

In [56]:
run_action(2019,3,6,15)

Execution completed in 593.584990978241 seconds


In [29]:
# from multiprocessing import Pool

# def get_url(i):
#     '''Basic worker function'''
#     p = 'https://www.python.org/events/python-events/' + str(i) + '/'
#     wget.download(p, 'index_'+ str(i)+ '_pool.html')
    


# if(__name__ == "__main__"):    
#     start_time = time.time()
#     # create pool of threads
#     pool = Pool(20)
#     #distribute the work to the threads
#     results = pool.map(get_url,range(1,201))
#     #wait for threads to finish
#     pool.close()
#     pool.join()
    
#     elapsed_2 = time.time()-start_time
#     print("Execution completed in {} seconds".format(elapsed_2))

MaybeEncodingError: Error sending result: '<multiprocessing.pool.ExceptionWithTraceback object at 0x7f0b1c324910>'. Reason: 'TypeError("cannot serialize '_io.BufferedReader' object")'

In [10]:
        def get_GIDs(q):
            '''
            Function to get GameIDs 
            Input: Queue of dates 
            Output: Queue and dictionary of the Gids  
            Note: Make sure the function of this directory is located in the 
            '''

            day = q.get()
            All_urls = []
            Return_URl = []
            while (day != 'Done'):
                All_urls.append('./MLB_Gameday_Data/year_' + str(day[0:4]) + '/month_'+ str(day[5:7]) + '/day_' + str(day[8:10]))      
                day = q.get()
            ID_dic = {}
            GID_queue = Queue()

            r = '(\w+)/\S\S(\w+_[0-9]{4}_[0-9]{2}_[0-9]{2}_\w+_\w+_[0-9])'
            for i in range(0,len(All_urls)):
                with open(All_urls[i] + '/index.html', 'r') as f:
                    try:
                        lines = f.readlines()
                        matches = re.finditer(r,str(lines))
                        for m in matches:
                            GID_queue.put(m.group(2))
                            ID_dic[m.group(1)] = m.group(2)
                            Return_URl.append(All_urls[i])
                    except:
                        pass
            GID_Search(ID_dic, GID_queue, All_urls)



        def GID_Search(Dict, GID, url):
            '''
            Function to ccheck GID and game values
            Input: Dictionary of game values and GIDs, a queue of GIDs 
            Output: Creation of a Directory with all the GIDs in them
            Note: Make sure the function of this directory is located in the 

            let us make url a list too

            '''
            home_dir = os.getcwd()
            MLB_GIDs =  Path('./MLB_GIDs')
            if MLB_GIDs.exists() is False:
                os.makedirs('./MLB_GIDs')
            Move_file = home_dir + '/MLB_GIDs'
            GID.put('Done')
            GameID = GID.get()
            urlNumber = 0
            while (GameID != 'Done' and urlNumber < len(url)):

                os.chdir(home_dir)
                os.chdir(url[urlNumber])
                #print(urlNumber)
                list_dir = os.listdir()
                
                urlNumber += 1
                try:
                    for i in list_dir:
                        Game_numbers = []
                        if i != 'index.html':
                            Game_numbers.append(i)
                            
                        for i in Game_numbers:
#                             print(i)
#                             File_name = Dict.get(i)
#                             print(File_name)
#                             print('______')
                            try:
                                shutil.copytree(i, Move_file + '/' + i)
                            except:
                                pass
                                #print('failed to move:' + i)


                except:
                    pass
            os.chdir(Move_file)
            new_dir = os.listdir()
            for i in new_dir:
                os.rename(i,Dict.get(i))

        def run_action(year1,month1,day1,threads):   
            start_time = time.time()
            Daysqueue = Queue()

            os.chdir('/home/brian/Data3402/Project1')
            start = date(year1,month1,day1)
            today = date.today()
            day = start
            while(day <= today):
                Daysqueue.put(day.isoformat())
                day += timedelta(1)
            Daysqueue.put('Done')

            num_threads = threads
            thrds = []
            for i in range(num_threads):
                p = Process(target = get_GIDs, args=[Daysqueue])
                thrds.append(p)
                p.start()

                #wait for threads to finish
            for p in thrds:
                p.join
            elapsed_6 = time.time()-start_time
            print("Execution completed in {} seconds".format(elapsed_6))

In [16]:
run_action(2019,1,4,10)

Execution completed in 0.1504533290863037 seconds


Process Process-153:
Process Process-159:
Process Process-154:
Process Process-158:
Process Process-160:
Process Process-152:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/brian/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/brian/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/brian/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/brian/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/brian/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/brian/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  Fi